In [29]:
# 1. Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from tensorflow import keras
from keras import layers
from keras_tuner import HyperModel, RandomSearch
from pypfopt import EfficientFrontier, risk_models, expected_returns
import json
import re
import pickle

In [30]:
global cleaned_weights

In [31]:
# with open('symbols.json', 'r') as f:
#    tickers = json.load(f)

# pk_filenames.json = ["data/ibm.pk1", "data/aapl.pk1"]

with open('pk_filenames.json','r') as f:
    filenames = json.load(f)

# for filename in filenames:
#     df = pd.read_pickle(filename)

In [32]:
# 2. Data Collection
# with open('symbols.json', 'r') as f:
#    tickers = json.load(f)

# def fetch_data(ticker, start, end):
#     data = yf.download(ticker, start=start, end=end)
#     return data['Adj Close']

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'TLT', 'COST', 'WMT', 'BA', 'DIS', 'JPM', 'AMD']
all_expected_returns = {}
all_data = pd.DataFrame()

In [41]:
# Loop through each ticker individually
# for ticker in tickers:
for filename in filenames:
    df = pd.read_pickle(filename)
    # data = fetch_data(ticker, '2020-01-01', '2023-01-01')
    data = df['Adj Close']
    ticker = filename.split('/')[1].split('.')[0]

    all_data[ticker] = data  # Store data for covariance calculation
    # 3. Data Preprocessing
    returns = data.pct_change().dropna()

    # Reshape data for LSTM in a compatible format
    X = returns.values.reshape(-1, 1, 1)

    # 4. LSTM Modeling
    class LSTMHyperModel(HyperModel):
        def build(self, hp):
            model = keras.Sequential()
            model.add(layers.Input(shape=(1, 1)))
            model.add(layers.LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), activation='relu'))
            model.add(layers.Dense(1))
            model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3])), loss='mse')
            return model

    # Hyperparameter tuning
    tuner = RandomSearch(
        LSTMHyperModel(),
        objective='val_loss',
        max_trials=5,
        executions_per_trial=3,
        directory='lstm_tuning',
        project_name=f'portfolio_optimization_{ticker}'
    )

    # Train the model only for the current ticker
    tuner.search(X, returns.values, epochs=10, validation_split=0.2)
    best_model = tuner.get_best_models(num_models=1)[0]

    # Predict future returns
    predicted_returns = best_model.predict(X)

    # Aggregate predictions to get expected daily returns for each asset
    expected_daily_return = np.mean(predicted_returns)

    # Annualize the expected daily return
    annualized_return = (1 + expected_daily_return) ** 252 - 1
    all_expected_returns[ticker] = annualized_return

    # Print annualized expected returns for each ticker
    print(f"Annualized Expected Returns for {ticker}: {annualized_return}")
    
    filename = ticker + '_annualized_return.pkl'
    with open(filename,'wb') as f:
        pickle.dump(annualized_return,f)
    

Reloading Tuner from lstm_tuning\portfolio_optimization_tsla\tuner0.json
 1/40 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step

c:\Users\16023\anaconda3\envs\algotrading\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Annualized Expected Returns for tsla: 1.074933771013197
Reloading Tuner from lstm_tuning\portfolio_optimization_v\tuner0.json
 1/40 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step

c:\Users\16023\anaconda3\envs\algotrading\lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Annualized Expected Returns for v: 0.1447416123930647


In [34]:
# Convert the dictionary to a Pandas Series
expected_returns_series = pd.Series(all_expected_returns)

In [35]:
# Portfolio Optimization
# Calculate the covariance matrix using all tickers' data
cov_matrix = risk_models.risk_matrix(all_data, method='ledoit_wolf' )
# https://pyportfolioopt.readthedocs.io/en/latest/RiskModels.html
ef = EfficientFrontier(expected_returns=expected_returns_series, cov_matrix=cov_matrix)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

# Print the optimized portfolio weights
print("Optimized Portfolio Weights:", cleaned_weights)

Optimized Portfolio Weights: OrderedDict([('tsla', 1.0), ('v', 0.0)])


In [36]:
with open('cleaned_weights.pkl','wb') as f:
    pickle.dump(cleaned_weights,f)

In [38]:
with open('cleaned_weights.pkl','rb') as f:
    load_dict = pickle.load(f)
    
print(load_dict)

OrderedDict([('tsla', 1.0), ('v', 0.0)])
